In [1]:
import pandas as pd
import datetime, json, requests

from brownie import web3, network, Wei, Contract, project
from brownie.network.account import LocalAccount

In [2]:
def telegram_bot_sendtext(bot_message):
    
    bot_token = '1262543569:AAEX0QVuvGpyooBG5R3Cztq1wwdaDAcZwQ4'
    bot_chatID = '-456518436'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

In [3]:
network_selected = "ropsten"
try: 
    network.connect(network_selected)
except: 
    network.connect(network_selected, launch_rpc=False)

In [4]:
## Admin account
fss_admin_private_key = "ede4dd8a3584fd7809a5e0bb299ff8f51983af5b1a9f1f506165b5c1f09e22b1"
fss_admin_account = web3.eth.account.from_key(private_key=fss_admin_private_key)
local_account_admin = LocalAccount(fss_admin_account.address, fss_admin_account, fss_admin_account.privateKey)

In [5]:
## Init team1 (CST) and team3 (AA) tokens, exchanges and the PayCoin

print("Opening the contract...")
with open('../../../../blockchain_course_unimi/challenge/teamCST/abi/Exchange.json') as json_file: 
    exchange_CST_abi = json.load(json_file)
exchange_CST = Contract.from_abi('ExchangeCST', address='0xf6595CF80173Edf534469B15170370AbFF3FDdAb', abi=exchange_CST_abi)

with open('../../../../blockchain_course_unimi/challenge/teamAA/abi/real/exchange.json') as json_file: 
    exchange_AA_abi = json.load(json_file)
exchange_AA = Contract.from_abi('ExchangeAA', address='0x5b349092f8F7A4f033743e064c61FDAea6629Db2', abi=exchange_AA_abi)
print("Finished opening.")

Opening the contract...
Finished opening.


In [6]:
df_CST = pd.read_csv(r'./token_prices.csv', sep='\t', index_col=0, usecols=[0, 1])
df_AA = pd.read_csv(r'./token_prices.csv', sep='\t',  index_col=0, usecols=[0, 2], nrows=8760)

In [13]:
df_CST.shape[0]

8790

In [17]:
df_AA.shape[0]

8760

In [10]:
prices_list_CST = []
prices_list_AA = []

In [11]:
exchange_CST.lastPrice()[0]

8838

In [12]:
exchange_AA.lastPrice()[0]

8803

In [14]:
for i in range(8791, exchange_CST.lastPrice()[0]+1): 
    try:
        prices_list_CST.append(exchange_CST.getHistory(i))
    except:
        print(i)
        print(Exception)
        continue

In [18]:
for i in range(8761, exchange_AA.lastPrice()[0]+1):
    try:
        prices_list_AA.append(exchange_AA.getHistory(i))
    except: 
        print(Exception)
        continue

In [19]:
df_CST_temp = pd.DataFrame(prices_list_CST, columns=['TokenCST'])
df_AA_temp = pd.DataFrame(prices_list_AA, columns=['TokenAA'])

In [22]:
print(df_CST.tail())
print(df_CST_temp)

                                 TokenCST
2019-06-22 22:00:00  20460000000000000000
2019-06-22 21:00:00  20420000000000000000
2019-06-22 20:00:00  20369999999999995904
2019-06-22 19:00:00  20330000000000000000
2019-06-22 18:00:00  20380000000000004096
                TokenCST
0   20380000000000004096
1   20330000000000000000
2   20280000000000000000
3   20350000000000000000
4   20260000000000000000
5   20190000000000000000
6   20170000000000000000
7   20230000000000000000
8   20170000000000000000
9   20060000000000000000
10  19840000000000000000
11  19970000000000000000
12  19990000000000000000
13  19940000000000000000
14  19890000000000000000
15  20050000000000000000
16  20050000000000000000
17  20120000000000000000
18  20070000000000000000
19  20100000000000000000
20  20050000000000000000
21  20160000000000000000
22  20090000000000000000
23  19790000000000000000
24  19620000000000000000
25  19610000000000000000
26  19570000000000000000
27  19380000000000004000
28  1942000000000000000

In [24]:
print(df_AA.head())
print(df_AA_temp)

                          TokenAA
2020-06-22 23:00:00  5.026900e+16
2020-06-22 22:00:00  5.030100e+16
2020-06-22 21:00:00  5.024900e+16
2020-06-22 20:00:00  5.018420e+16
2020-06-22 19:00:00  5.025000e+16
               TokenAA
0   159858500000000000
1   157018300000000000
2   152327500000000000
3   152227900000000000
4   154706999999999968
5   158770000000000000
6   157067100000000000
7   159511100000000032
8   162268000000000000
9   160556999999999968
10  160546900000000000
11  163579999999999968
12  164350000000000000
13  167364000000000000
14  171900000000000000
15  171083100000000000
16  171246999999999968
17  167323900000000000
18  165477899999999968
19  165740000000000000
20  166299900000000000
21  171077999999999968
22  169281000000000000
23  170000000000000000
24  176366000000000000
25  175863800000000000
26  173900000000000000
27  179107600000000000
28  178000000000000032
29  181299900000000000
30  183208000000000000
31  180996900000000032
32  180369000000000000
33  1807989000

In [25]:
def datetime_range(start, end, delta):
    current = start
    if not isinstance(delta, datetime.timedelta):
        delta = datetime.timedelta(**delta)
    while current < end:
        yield current
        current += delta

In [29]:
start = datetime.datetime(2019,6,22)
end = datetime.datetime(2020,6,24)

In [30]:
dates = []
dates_refined = []

In [31]:
for dt in datetime_range(start, end, {'hours':1}):
    dates.append(dt)

In [32]:
for i in range(1, df_CST.shape[0]+ 1): 
    dates_refined.append(dates[-i])

In [37]:
df_CST.index = dates_refined

In [38]:
df_CST

,TokenCST
2019-06-23 18:00:00,15730000000000000000
2019-06-23 19:00:00,15600000000000000000
2019-06-23 20:00:00,15919999999999997952
2019-06-23 21:00:00,16180000000000000000
2019-06-23 22:00:00,16020000000000000000
...,...
2020-06-23 19:00:00,20460000000000000000
2020-06-23 20:00:00,20420000000000000000
2020-06-23 21:00:00,20369999999999995904
2020-06-23 22:00:00,20330000000000000000


In [39]:
dates_refined = []
for i in range(1,  df_AA.shape[0]+ 1): 
    dates_refined.append(dates[-i])

In [41]:
dates_refined.reverse()
dates_refined

[datetime.datetime(2019, 6, 25, 0, 0),
 datetime.datetime(2019, 6, 25, 1, 0),
 datetime.datetime(2019, 6, 25, 2, 0),
 datetime.datetime(2019, 6, 25, 3, 0),
 datetime.datetime(2019, 6, 25, 4, 0),
 datetime.datetime(2019, 6, 25, 5, 0),
 datetime.datetime(2019, 6, 25, 6, 0),
 datetime.datetime(2019, 6, 25, 7, 0),
 datetime.datetime(2019, 6, 25, 8, 0),
 datetime.datetime(2019, 6, 25, 9, 0),
 datetime.datetime(2019, 6, 25, 10, 0),
 datetime.datetime(2019, 6, 25, 11, 0),
 datetime.datetime(2019, 6, 25, 12, 0),
 datetime.datetime(2019, 6, 25, 13, 0),
 datetime.datetime(2019, 6, 25, 14, 0),
 datetime.datetime(2019, 6, 25, 15, 0),
 datetime.datetime(2019, 6, 25, 16, 0),
 datetime.datetime(2019, 6, 25, 17, 0),
 datetime.datetime(2019, 6, 25, 18, 0),
 datetime.datetime(2019, 6, 25, 19, 0),
 datetime.datetime(2019, 6, 25, 20, 0),
 datetime.datetime(2019, 6, 25, 21, 0),
 datetime.datetime(2019, 6, 25, 22, 0),
 datetime.datetime(2019, 6, 25, 23, 0),
 datetime.datetime(2019, 6, 26, 0, 0),
 datetime.d

In [42]:
df_AA.index = dates_refined

In [43]:
df_AA

,TokenAA
2019-06-25 00:00:00,5.026900e+16
2019-06-25 01:00:00,5.030100e+16
2019-06-25 02:00:00,5.024900e+16
2019-06-25 03:00:00,5.018420e+16
2019-06-25 04:00:00,5.025000e+16
...,...
2020-06-23 19:00:00,1.514530e+17
2020-06-23 20:00:00,1.514530e+17
2020-06-23 21:00:00,1.571833e+17
2020-06-23 22:00:00,1.551635e+17


In [77]:
start = datetime.datetime(2020,6,23,23)
end = datetime.datetime(2020,6,24,15)

In [83]:
dates = []
dates_refined = []

In [84]:
for dt in datetime_range(start, end, {'seconds': 900}):
    dates.append(dt)

In [85]:
len(dates)

64

In [86]:
for i in range(1, 45): 
    dates_refined.append(dates[-i])

In [87]:
dates_refined.reverse()
dates_refined

[datetime.datetime(2020, 6, 24, 4, 0),
 datetime.datetime(2020, 6, 24, 4, 15),
 datetime.datetime(2020, 6, 24, 4, 30),
 datetime.datetime(2020, 6, 24, 4, 45),
 datetime.datetime(2020, 6, 24, 5, 0),
 datetime.datetime(2020, 6, 24, 5, 15),
 datetime.datetime(2020, 6, 24, 5, 30),
 datetime.datetime(2020, 6, 24, 5, 45),
 datetime.datetime(2020, 6, 24, 6, 0),
 datetime.datetime(2020, 6, 24, 6, 15),
 datetime.datetime(2020, 6, 24, 6, 30),
 datetime.datetime(2020, 6, 24, 6, 45),
 datetime.datetime(2020, 6, 24, 7, 0),
 datetime.datetime(2020, 6, 24, 7, 15),
 datetime.datetime(2020, 6, 24, 7, 30),
 datetime.datetime(2020, 6, 24, 7, 45),
 datetime.datetime(2020, 6, 24, 8, 0),
 datetime.datetime(2020, 6, 24, 8, 15),
 datetime.datetime(2020, 6, 24, 8, 30),
 datetime.datetime(2020, 6, 24, 8, 45),
 datetime.datetime(2020, 6, 24, 9, 0),
 datetime.datetime(2020, 6, 24, 9, 15),
 datetime.datetime(2020, 6, 24, 9, 30),
 datetime.datetime(2020, 6, 24, 9, 45),
 datetime.datetime(2020, 6, 24, 10, 0),
 datet

In [88]:
df_AA_temp.index = dates_refined

In [90]:
len(prices_list_CST)

48

In [91]:
dates_refined = []
for i in range(1, 49): 
    dates_refined.append(dates[-i])

In [101]:
dates_refined.reverse()

In [102]:
df_CST_temp.index = dates_refined

In [103]:
df_CST_1 = pd.concat([df_CST, df_CST_temp])
df_CST_1

,TokenCST
2019-06-23 18:00:00,15730000000000000000
2019-06-23 19:00:00,15600000000000000000
2019-06-23 20:00:00,15919999999999997952
2019-06-23 21:00:00,16180000000000000000
2019-06-23 22:00:00,16020000000000000000
...,...
2020-06-24 13:45:00,21150000000000000000
2020-06-24 14:00:00,21120000000000000000
2020-06-24 14:15:00,21490000000000000000
2020-06-24 14:30:00,21100000000000000000


In [104]:
df_AA_1 = pd.concat([df_AA, df_AA_temp])
df_AA_1

,TokenAA
2019-06-25 00:00:00,5.026900e+16
2019-06-25 01:00:00,5.030100e+16
2019-06-25 02:00:00,5.024900e+16
2019-06-25 03:00:00,5.018420e+16
2019-06-25 04:00:00,5.025000e+16
...,...
2020-06-24 13:45:00,1.851450e+17
2020-06-24 14:00:00,1.857010e+17
2020-06-24 14:15:00,1.853808e+17
2020-06-24 14:30:00,1.892000e+17


In [105]:
df_final = df_CST_1.join(df_AA_1)

In [107]:
df_final.to_csv('./token_prices.csv', sep='\t')